In [1]:
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
class RBFlayer(nn.Module):
    def __init__(self, timelag):
        super(RBFlayer, self).__init__()

        self.timelag = timelag

        # device = "cuda" if torch.cuda.is_available() else "cpu"
        torch.cuda.manual_seed(0)

        self.init_weight = nn.Parameter(torch.rand(self.timelag))
        self.rbf_clt = self.init_clt()
        self.rbf_std = self.init_std()
        
        self.b = nn.Parameter(torch.rand(1))

    def init_clt(self):
        return nn.Parameter(torch.rand(1))

    def init_std(self):
        return nn.Parameter(torch.rand(1))
    
    def rbf(self, x, cluster, std):
        return torch.exp(-(x - cluster) * (x - cluster) / 2 * (std * std))
    
    def rbf_gradient(self, x, clt, std):
        return (-1 * (x - clt) * (x - clt) / (std * std)) * (torch.exp(-(x - clt) * (x - clt) / 2 * (std * std)))
    
    
    def forward(self, x):        
        for i in range(len(x)):
            if i == 0:
                a = self.rbf(x[i], self.rbf_clt, self.rbf_std)
            else:
                a = torch.cat([a, self.rbf(x[i], self.rbf_clt, self.rbf_std)], dim=0)
        cause = self.init_weight * a
        
        
        return cause

In [3]:
def restore_parameters(model, best_model):
    '''Move parameter values from best_model to model.'''
    for params, best_params in zip(model.parameters(), best_model.parameters()):
        params.data = best_params

In [12]:
def train_RBFlayer(model, input_, lr, epochs, lookback = 5, device = device):
    model.to(device)
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    
    train_loss_list = []
    
    best_it = None
    best_model = None
    best_loss = np.inf
    target = []
    for j in range(len(input_) - 2):
        target.append((input_[j+2] - input_[j])/2)
    
    loss_list = []
    cause_list = []
    for epoch in range(epochs):
        cause = model(input_)
        cause_list.append(cause)
        grad = []
        
        
        for i in range(len(cause) - 2):
            grad.append((cause[i+2] - cause[i])/2)
        
        loss = sum([loss_fn(grad[i], target[i]) for i in range(len(grad))])

        '''
        print("epoch {} cause loss {} :".format(epoch, loss / len(input_)))
        print("------------------------------------------------------")
        print()
        '''
        loss.backward()
        optimizer.step()
        model.zero_grad()
        
        loss_list.append(loss)
        mean_loss = loss / len(grad)
        train_loss_list.append(mean_loss)
        
        if mean_loss < best_loss:
            best_loss = mean_loss
            best_it = epoch
            best_model = deepcopy(model)
            
        elif (epoch - best_it) == lookback:
            if verbose:
                print('Stopping early')
            break
    print("epoch {} cause loss {} :".format(epoch, loss / len(input_)))
                
    best_cause = cause_list[best_it]    
    restore_parameters(model, best_model)

    return best_model, loss_list, best_cause

In [5]:
def data_split(X, cause, target, timelag, device = device):
    input_cause = []
    input_target = []
    Y = []

    for i in range(len(X) - (timelag + 1)):
        input_cause.append(X[cause].values[i: i + timelag])
        input_target.append(X[target].values[i: i + timelag])
        Y.append([X[target][i + timelag + 1]])

    return torch.tensor(input_cause, device=device).float(), torch.tensor(input_target,device=device).float(), torch.tensor(Y, device=device).float()


In [6]:
import pandas as pd
df = pd.read_csv('C:/Users/chanyoung/Desktop/Neural-GC-master/lorenz_96_10_10_1000.csv')
X2d = df[['a','b']]
torch.manual_seed(1234)
input_cause, input_target, Y = data_split(X2d, 'a', 'b', 100)

In [7]:
input_cause = input_cause[:20]
input_cause.size()

In [13]:
import time

cause_list = []
for i in range(len(input_cause)):
    print(i,"번째 time series")
    start = time.time()
    model = RBFlayer(100)
    best_model, loss_list, best_cause = train_RBFlayer(model, input_cause[i], 0.01, 1000, device)
    cause_list.append(best_cause.cpu().detach().numpy())
    print("time :", time.time() - start)
    print('-------------------------------------------------------------------------------------------')

0 번째 time series
epoch 999 cause loss 0.061188917607069016 :
time : 115.37664747238159
-------------------------------------------------------------------------------------------
1 번째 time series
epoch 999 cause loss 0.027772853150963783 :
time : 113.19796395301819
-------------------------------------------------------------------------------------------
2 번째 time series
epoch 999 cause loss 0.03972911089658737 :
time : 112.32275438308716
-------------------------------------------------------------------------------------------
3 번째 time series
epoch 999 cause loss 0.020239241421222687 :
time : 125.69012117385864
-------------------------------------------------------------------------------------------
4 번째 time series
epoch 999 cause loss 0.026726502925157547 :
time : 121.11877536773682
-------------------------------------------------------------------------------------------
5 번째 time series
epoch 999 cause loss 0.019136741757392883 :
time : 122.63941359519958
-------------------

In [14]:
import pickle

filePath = './test3_epcoh1000.txt'
with open(filePath, 'wb') as lf:
    pickle.dump(cause_list, lf)

In [9]:
cause_list = []
for i in range(len(input_cause)):
    print(i,"번째 time series")
    model = RBFlayer(100)
    best_model, loss_list, best_cause = train_RBFlayer(model, input_cause[i], 0.01, 2000, device)
    cause_list.append(best_cause.cpu().detach().numpy())
    
    


filePath = './test2_epcoh2000.txt'
with open(filePath, 'wb') as lf:
    pickle.dump(cause_list, lf)

0 번째 time series
epoch 1999 cause loss 0.0011501103872433305 :
------------------------------------------------------
1 번째 time series
epoch 1999 cause loss 0.0015078928554430604 :
------------------------------------------------------
2 번째 time series
epoch 1999 cause loss 0.0011376310139894485 :
------------------------------------------------------
3 번째 time series
epoch 1999 cause loss 0.0007262340513989329 :
------------------------------------------------------
4 번째 time series
epoch 1999 cause loss 0.00048040205729193985 :
------------------------------------------------------
5 번째 time series
epoch 1999 cause loss 0.0007808118825778365 :
------------------------------------------------------
6 번째 time series
epoch 1999 cause loss 0.0013475314481183887 :
------------------------------------------------------
7 번째 time series
epoch 1999 cause loss 0.0007810455863364041 :
------------------------------------------------------
8 번째 time series
epoch 1999 cause loss 0.00078795023728